In [6]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# For webcam input:
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture('datasets/data_corr_incorr/Correct sequence/Copy of push up 2.mp4')  # put your video path here

state = "UP"
up_frames = []
down_frames = []

frame_count = 0
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Video file finished or failed.")
        break

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        # getting the landmark coordinates for shoulder, hip and ankle
        r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
        # Calculating angle between the lines - Shoulder-Hip and Hip-Ankle
        angle = calculate_angle(r_shoulder, r_hip, r_ankle)
        
        # If angle > 160 - Person is in UP position
        if angle > 160:
            state = "UP"
            up_frames.append(frame_count)
        else: 
            # If angle < 100 - Person is in Down position
            if angle < 100 and state == 'UP':
                state = "DOWN"
                down_frames.append(frame_count)
    
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

print("UP frames: ", up_frames)
print("DOWN frames: ", down_frames)


Video file finished or failed.
UP frames:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
DOWN frames:  []


In [9]:
# we have our pushups dataset in the form of video, we now want to extract the frames where the person is in the up position and the down position.
def get_up_down_positions(video_path):
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)

    state = "UP"
    up_frames = []
    down_frames = []

    frame_count = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Video file finished or failed.")
            break

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            # getting the landmark coordinates for shoulder, hip and ankle
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculating angle between the lines - Shoulder-Hip and Hip-Ankle
            angle = calculate_angle(r_shoulder, r_hip, r_ankle)

            # If angle > 160 - Person is in UP position
            if angle > 160:
                state = "UP"
                up_frames.append(frame_count)
            else: 
                # If angle < 100 - Person is in Down position
                if angle < 100 and state == 'UP':
                    state = "DOWN"
                    down_frames.append(frame_count)

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    return up_frames, down_frames

up_frames, down_frames = get_up_down_positions('datasets/data_corr_incorr/Correct sequence/Copy of push up 2.mp4')

Video file finished or failed.
